In [2]:
import pandas as pd

questions_df = pd.read_csv("../data/questions.csv")


In [10]:
e_q_df = pd.read_excel("../data/questions_src/srcs.xlsx", sheet_name=0)
e_r_df = pd.read_excel("../data/questions_src/srcs.xlsx", sheet_name=3)

s_q_df = pd.read_excel("../data/questions_src/srcs.xlsx", sheet_name=1)
s_r_df = pd.read_excel("../data/questions_src/srcs.xlsx", sheet_name=4)

g_q_df = pd.read_excel("../data/questions_src/srcs.xlsx", sheet_name=2)
g_r_df = pd.read_excel("../data/questions_src/srcs.xlsx", sheet_name=5)

In [36]:
import pandas as pd
import re

def transform_df(df, category):
    new_rows = []
    current_num = None
    current_question = None
    current_choices = []
    category_str = category

    def flush():
        if current_num is not None:
            int_current_num = int(current_num)
            new_rows.append({
                'question_number': int_current_num,
                'question': current_question,
                'choices': '\n'.join(current_choices),
                'category': category_str
            })

    for _, row in df.iterrows():
        number = row['번호']
        question_text = str(row['평가항목']) if pd.notna(row['평가항목']) else ""

        # 새로운 질문 발견 시 처리
        if pd.notna(number):
            flush()
            current_num = number
            current_question = question_text
            current_choices = []
        else:
            # 가. 나. 다. 형태의 choices 감지
            if re.match(r'^[가나다라마바사아자차카타파하]\.\s', question_text.strip()):
                current_choices.append(question_text.strip())

    flush()
    return pd.DataFrame(new_rows, columns=['question_number','question','choices', 'category'])

In [ ]:
def merge_ref_to_questions(df1, df2):
    grouped = df2.groupby('항목 번호').agg({
        '대분류': 'first',
        '중분류': 'first',
        '정의 및 확인사항 + 참고사례 및 제출 시 유의사항': lambda x: ' '.join(x.dropna())
    }).reset_index()

    grouped.rename(columns={'정의 및 확인사항 + 참고사례 및 제출 시 유의사항': '참고사항'}, inplace=True)

    merged_df = pd.merge(
        df1, 
        grouped, 
        left_on='question_number', 
        right_on='항목 번호', 
        how='left'
    )
    
    return merged_df.drop(columns=['항목 번호'])


In [56]:
e_questions_wich_choices = transform_df(e_q_df, "E")
merged_e_questions_wich_choices = merge_ref_to_questions(e_questions_wich_choices, e_r_df)

s_questions_wich_choices = transform_df(s_q_df, "S")
merged_s_questions_wich_choices = merge_ref_to_questions(s_questions_wich_choices, s_r_df)

g_questions_wich_choices = transform_df(g_q_df, "G")
merged_g_questions_wich_choices = merge_ref_to_questions(g_questions_wich_choices, g_r_df)

In [59]:
pd.concat([merged_e_questions_wich_choices, merged_s_questions_wich_choices, merged_g_questions_wich_choices]).to_csv("../data/questions_with_ref.csv", index=False)

In [61]:
pd.read_csv("../data/questions_with_ref.csv")

,question_number,question,choices,category,대분류,중분류,참고사항
0,1,이사회 내에서 기후변화 및 탄소중립 안건을 보고하거나 결의하였는가?,가. 아니오\n나. 안건보고\n다. 안건결의,E,리더십과 거버넌스,거버넌스,-\n\n- (정의)\n기후변화 및 탄소중립 이슈: 온실가스 및 에너지 등 기후변화...
1,2,이사회 내에서 그 밖의* 주요한 환경경영 안건을 보고하거나 결의하였는가? (*자원순...,가. 아니오\n나. 안건보고\n다. 안건결의,E,리더십과 거버넌스,거버넌스,-\n\n- (정의)\n그 밖의 주요한 환경이슈: 기후변화 및 탄소중립으로 특정되지...
2,3,이사회 내에서 정기적으로 환경경영 성과를 검토하고 있는가?,가. 아니오\n나. 비정기적\n다. 정기적,E,리더십과 거버넌스,거버넌스,-\n\n- (정의)\n1. 정기적: 아래 두 가지 요건 중 하나 이상 충족 필요 ...
3,4,최고경영진의 환경경영 관련 책임 및 권한을 규정하고 명문화하였는가?,가. 아니오\n나. 경영진\n다. 최고경영진(C-Level),E,리더십과 거버넌스,거버넌스,-\n\n- (정의)\n1. 최고경영진(C-Level 임원): 직무 부문별 최고책임...
4,5,환경경영 이행 전담조직을 구축하고 역할과 책임을 공개하고 있는가?,가. 아니오\n나. 예,E,리더십과 거버넌스,거버넌스,(확인사항)\n조직도 내 환경경영 전담 조직명 공개 혹은 환경경영 담당팀/담당자의 ...
...,...,...,...,...,...,...,...
162,62,최근 정기주주총회의 의결권 행사 현황*을 홈페이지에 공개하고 있는가? * 참석 의결...,가. 아니오\n나. 예,G,이해관계자 소통,정보공개,(확인사항) \n- 최근 정기주주총회의 참석 의결권 수 홈페이지 공개 여부 \n\n...
163,63,감사보고서 또는 사업보고서를 영문으로 공개하는가?,가. 아니오\n나. 예,G,이해관계자 소통,정보공개,(확인사항)\n- 최근 2개 사업연도까지의 영문 감사보고서 대외 공개 여부\n- 최...
164,64,"최근 E,S,G 개별 등급 및 통합 ESG 평가등급에 관한 정보를 홈페이지에 공개하...",가. 아니오\n나. 예,G,이해관계자 소통,정보공개,"(확인사항) \n- 지배구조(G)평가등급 홈페이지 공개 여부 \n- 평가기관명, 평..."
165,65,"최근 5개년의 배당정보*를 홈페이지에 공개하고 있는가? * 배당총액, 주당 배당금,...",가. 아니오\n나. 예,G,이해관계자 소통,정보공개,(정의) \n\n- 배당총액: 주식 총 발행 수량x주당배당금 \n\n\n\n(확인사...


In [62]:
pd.read_csv("../data/reports/2024/현대차/graded.csv")

,항목번호,question,choices,grade,explanation,retrieved_page_numbers
0,1,이사회 내에서 기후변화 및 탄소중립 안건을 보고하거나 결의하였는가?\n \n\n 아...,가. 아니오\n나. 안건보고\n다. 안건결의,다. 안건결의,[단계별 검토]\n1) 보고서 내 기후변화 관련 이사회(또는 이사회 산하 위원회) ...,"[21, 36, 17]"
1,2,이사회 내에서 그 밖의* 주요한 환경경영 안건을 보고하거나 결의하였는가?\n (*자...,가. 아니오\n나. 안건보고\n다. 안건결의,가. 아니오,(1) 첨부된 보고서(특히 p.85 ‘2023년 지속가능경영위원회 개최 현황’)를 ...,"[10, 16, 85]"
2,3,이사회 내에서 정기적으로 환경경영 성과를 검토하고 있는가?\n \n\n 아래 항목들...,가. 아니오\n나. 비정기적\n다. 정기적,다. 정기적,보고서 내에서 기후변화 위기 관리 프로세스를 비롯해 이사회 산하 ESG위원회 및 경...,"[36, 115, 110]"
3,4,최고경영진의 환경경영 관련 책임 및 권한을 규정하고 명문화하였는가?\n \n\n 아...,가. 아니오\n나. 경영진\n다. 최고경영진(C-Level),다. 최고경영진(C-Level),[풀이과정]\n1) 보고서 내 ‘지속가능경영 거버넌스’(9p) 및 ‘환경경영 체계’...,"[19, 110, 9]"
4,5,환경경영 이행 전담조직을 구축하고 역할과 책임을 공개하고 있는가?\n \n\n 아래...,가. 아니오\n나. 예,가. 아니오,이미지 내 언급된 내용(예: 10페이지 등)을 보면 ‘ESG 전담 조직(ESG기획팀...,"[10, 100, 110]"
...,...,...,...,...,...,...
91,32,지역사회 참여와 관련하여 중장기 목표를 공개하고 있는가?\n \n\n 아래 항목들을...,가. 아니오\n나. 예,가. 아니오,보고서 내 지역사회 참여 관련 항목을 보면 회사의 이해관계자 그룹 정의와 소통 방향...,"[12, 10, 124]"
92,33,지역사회와의 상생을 위한 활동을 수행하고 성과를 공개하고 있는가?\n \n\n 아래...,가. 아니오\n나. 지역사회 상생 활동 공개\n다. 지역사회 상생 활동의 성과 공개,나. 지역사회 상생 활동 공개,"[1단계] ESG Factbook 104페이지(이미지상 ""지표 및 데이터"")에서 현...","[104, 77, 47]"
93,34,"주요 비재무 정보 공개를 위한 채널을 마련하고, 공개한 정보에 대해서는 제3자 검증...",가. 채널 없음\n나. 채널 마련 및 정보 공개\n다. 비재무정보에 대한 제3자 검...,다. 비재무정보에 대한 제3자 검증 실시,1) 보고서 내용에 따르면 현대자동차는 ‘Road to Sustainability’...,"[124, 117, 118]"
94,53,리스크 관리 책임자가 이사회에 리스크 관리활동을 보고하고 있는가?\n \n\n 아래...,가. 아니오\n나. 이사회에 보고\n다. 리스크관리 책임자가 이사회에 보고,다. 리스크관리 책임자가 이사회에 보고,[판단 근거]\n제공된 보고서 자료(특히 이미지 p.90 상단 부분)에서 현대자동차...,"[90, 91, 83]"


,question_number,question,choices,category,대분류,중분류,참고사항
0,1,이사회 내에서 기후변화 및 탄소중립 안건을 보고하거나 결의하였는가?,가. 아니오\n나. 안건보고\n다. 안건결의,E,리더십과 거버넌스,거버넌스,-\n\n- (정의)\n기후변화 및 탄소중립 이슈: 온실가스 및 에너지 등 기후변화...
1,2,이사회 내에서 그 밖의* 주요한 환경경영 안건을 보고하거나 결의하였는가? (*자원순...,가. 아니오\n나. 안건보고\n다. 안건결의,E,리더십과 거버넌스,거버넌스,-\n\n- (정의)\n그 밖의 주요한 환경이슈: 기후변화 및 탄소중립으로 특정되지...
2,3,이사회 내에서 정기적으로 환경경영 성과를 검토하고 있는가?,가. 아니오\n나. 비정기적\n다. 정기적,E,리더십과 거버넌스,거버넌스,-\n\n- (정의)\n1. 정기적: 아래 두 가지 요건 중 하나 이상 충족 필요 ...
3,4,최고경영진의 환경경영 관련 책임 및 권한을 규정하고 명문화하였는가?,가. 아니오\n나. 경영진\n다. 최고경영진(C-Level),E,리더십과 거버넌스,거버넌스,-\n\n- (정의)\n1. 최고경영진(C-Level 임원): 직무 부문별 최고책임...
4,5,환경경영 이행 전담조직을 구축하고 역할과 책임을 공개하고 있는가?,가. 아니오\n나. 예,E,리더십과 거버넌스,거버넌스,(확인사항)\n조직도 내 환경경영 전담 조직명 공개 혹은 환경경영 담당팀/담당자의 ...
...,...,...,...,...,...,...,...
162,62,최근 정기주주총회의 의결권 행사 현황*을 홈페이지에 공개하고 있는가? * 참석 의결...,가. 아니오\n나. 예,G,이해관계자 소통,정보공개,(확인사항) \n- 최근 정기주주총회의 참석 의결권 수 홈페이지 공개 여부 \n\n...
163,63,감사보고서 또는 사업보고서를 영문으로 공개하는가?,가. 아니오\n나. 예,G,이해관계자 소통,정보공개,(확인사항)\n- 최근 2개 사업연도까지의 영문 감사보고서 대외 공개 여부\n- 최...
164,64,"최근 E,S,G 개별 등급 및 통합 ESG 평가등급에 관한 정보를 홈페이지에 공개하...",가. 아니오\n나. 예,G,이해관계자 소통,정보공개,"(확인사항) \n- 지배구조(G)평가등급 홈페이지 공개 여부 \n- 평가기관명, 평..."
165,65,"최근 5개년의 배당정보*를 홈페이지에 공개하고 있는가? * 배당총액, 주당 배당금,...",가. 아니오\n나. 예,G,이해관계자 소통,정보공개,(정의) \n\n- 배당총액: 주식 총 발행 수량x주당배당금 \n\n\n\n(확인사...


In [56]:
import pandas as pd
import re

graded_path = "../data/reports/2023/종근당홀딩스/graded.csv"


q_df = pd.read_csv("../data/questions_with_ref.csv")
g_df = pd.read_csv(graded_path)
g_df

,question_number,question,choices,category,대분류,중분류,참고사항,grade,explanation,retrieved_page_numbers
0,1,이사회 내에서 기후변화 및 탄소중립 안건을 보고하거나 결의하였는가?,가. 아니오\n나. 안건보고\n다. 안건결의,E,리더십과 거버넌스,거버넌스,-\n\n- (정의)\n기후변화 및 탄소중립 이슈: 온실가스 및 에너지 등 기후변화...,가. 아니오,제공된 지속가능경영보고서의 텍스트와 이미지를 살펴보면 이사회에서 기후변화 또는 탄소...,"[39, 30, 28]"
1,2,이사회 내에서 그 밖의* 주요한 환경경영 안건을 보고하거나 결의하였는가? (*자원순...,가. 아니오\n나. 안건보고\n다. 안건결의,E,리더십과 거버넌스,거버넌스,-\n\n- (정의)\n그 밖의 주요한 환경이슈: 기후변화 및 탄소중립으로 특정되지...,NaN,NaN,NaN
2,3,이사회 내에서 정기적으로 환경경영 성과를 검토하고 있는가?,가. 아니오\n나. 비정기적\n다. 정기적,E,리더십과 거버넌스,거버넌스,-\n\n- (정의)\n1. 정기적: 아래 두 가지 요건 중 하나 이상 충족 필요 ...,다. 정기적,1) 2023 종근당홀딩스 지속가능경영보고서 35페이지(Material Topic ...,"[35, 69, 68]"
3,4,최고경영진의 환경경영 관련 책임 및 권한을 규정하고 명문화하였는가?,가. 아니오\n나. 경영진\n다. 최고경영진(C-Level),E,리더십과 거버넌스,거버넌스,-\n\n- (정의)\n1. 최고경영진(C-Level 임원): 직무 부문별 최고책임...,다. 최고경영진(C-Level),[1단계] 보고서 내용 중 ‘환경경영시스템 구축(General Topic–환경 파트...,"[68, 35, 5]"
4,5,환경경영 이행 전담조직을 구축하고 역할과 책임을 공개하고 있는가?,가. 아니오\n나. 예,E,리더십과 거버넌스,거버넌스,(확인사항)\n조직도 내 환경경영 전담 조직명 공개 혹은 환경경영 담당팀/담당자의 ...,나. 예,"[해설 단계별 접근]\n1) ""2023 종근당홀딩스 지속가능경영보고서""의 36페이지...","[68, 35, 36]"
...,...,...,...,...,...,...,...,...,...,...
162,62,최근 정기주주총회의 의결권 행사 현황*을 홈페이지에 공개하고 있는가? * 참석 의결...,가. 아니오\n나. 예,G,이해관계자 소통,정보공개,(확인사항) \n- 최근 정기주주총회의 참석 의결권 수 홈페이지 공개 여부 \n\n...,NaN,NaN,NaN
163,63,감사보고서 또는 사업보고서를 영문으로 공개하는가?,가. 아니오\n나. 예,G,이해관계자 소통,정보공개,(확인사항)\n- 최근 2개 사업연도까지의 영문 감사보고서 대외 공개 여부\n- 최...,NaN,NaN,NaN
164,64,"최근 E,S,G 개별 등급 및 통합 ESG 평가등급에 관한 정보를 홈페이지에 공개하...",가. 아니오\n나. 예,G,이해관계자 소통,정보공개,"(확인사항) \n- 지배구조(G)평가등급 홈페이지 공개 여부 \n- 평가기관명, 평...",NaN,NaN,NaN
165,65,"최근 5개년의 배당정보*를 홈페이지에 공개하고 있는가? * 배당총액, 주당 배당금,...",가. 아니오\n나. 예,G,이해관계자 소통,정보공개,(정의) \n\n- 배당총액: 주식 총 발행 수량x주당배당금 \n\n\n\n(확인사...,NaN,NaN,NaN


In [55]:
for i in range(len(q_df)):
    question  = q_df.iloc[i,1]
    escaped_string = re.escape(question)

    result_row = g_df[g_df['question'].str.contains(escaped_string, case=False, na=False)]

    if not result_row.empty:
        q_df.loc[i, 'grade'] =  result_row.iloc[0]['grade']
        q_df.loc[i, 'explanation'] =  result_row.iloc[0]['explanation']
        q_df.loc[i, 'retrieved_page_numbers'] =  result_row.iloc[0]['retrieved_page_numbers']
    else:
        continue

q_df.to_csv(graded_path, index=False)

In [60]:
g_df

if pd.isna(g_df.loc[1, 'grade']):
    print("NA")

NA


In [61]:
q_df = pd.read_csv("../data/questions_with_ref.csv")
q_df.loc[65:70]

,question_number,question,choices,category,대분류,중분류,참고사항
65,66,환경성과 공개 사업장 범위를 명시하고 있는가?,가. 아니오\n나. 예,E,이해관계자 소통,환경정보 공개,"(정의)\n1. 환경성과: 온실가스, 에너지, 폐기물, 용수, 오염물질 등 사업 활..."
66,67,"환경법규 위반 내역, 사고 재발방지 조치에 관한 사항 등을 공개하고 있는가?",가. 아니오\n나. 예,E,이해관계자 소통,환경정보 공개,(확인사항)\n평가대상기간(또는 평가대상기간을 포함한 최근 3년간) 내 국내외 환경...
67,1,영업 활동 또는 제품 및 서비스가 창출하는 사회적 가치를 공개하고 실적을 관리하고 ...,가. 아니오\n나. 사회적 가치 공개\n다. 사회적 가치 개선 실적 공개,S,리더십과 거버넌스,전략과 방침,NaN
68,2,근로자 다양성과 관련된 정보를 공개하고 있는가?,가. 아니오\n나. 직급별 구분 없이 공개\n다. 직급별로 공개,S,노동관행,공정하고 차별없는 고용,"(정의)\n근로자 다양성 : 성별, 장애여부, 국적, 연령, 학력, 성적지향 등과 ..."
69,3,관리자 이상 직급별 여성 임직원 비율을 공개하고 있는가?,가. 아니오\n나. 예,S,노동관행,공정하고 차별없는 고용,"(정의)\n관리자 : 일반적으로 중간관리자, 임원 등을 말하며 관리자에 대한 기준은..."
70,4,근로자 다양성 제고를 위한 목표를 공개하고 있는가?,가. 아니오\n나. 예,S,노동관행,공정하고 차별없는 고용,(정의)\n1. 근로자 다양성과 관련된 정량적 목표\n\n2. 근로자 다양성 : 성...


In [86]:

graded_path = "../data/reports/2024/삼성전자/graded.csv"

g_df = pd.read_csv(graded_path)
g_df


,question_number,question,choices,category,대분류,중분류,참고사항,grade,explanation,retrieved_page_numbers,rewrited_explanation_md
0,1.0,이사회 내에서 기후변화 및 탄소중립 안건을 보고하거나 결의하였는가?,가. 아니오\n나. 안건보고\n다. 안건결의,E,리더십과 거버넌스,거버넌스,-\n\n- (정의)\n기후변화 및 탄소중립 이슈: 온실가스 및 에너지 등 기후변화...,다. 안건결의,[단계별 검토]\n1) 보고서 내 기후변화 관련 이사회(또는 이사회 산하 위원회) ...,"[21, 36, 17]",## 요약\n- 등급: 다. 안건결의\n- 이사회 내에서 기후변화 및 탄소중립 관련...
1,2.0,이사회 내에서 그 밖의* 주요한 환경경영 안건을 보고하거나 결의하였는가? (*자원순...,가. 아니오\n나. 안건보고\n다. 안건결의,E,리더십과 거버넌스,거버넌스,-\n\n- (정의)\n그 밖의 주요한 환경이슈: 기후변화 및 탄소중립으로 특정되지...,가. 아니오,"보고된 정보에서는 기후변화나 탄소중립 외의 기타 환경이슈(예: 자원순환, 물ㆍ토양ㆍ...","[10, 16, 17]",## 요약 \n- 등급: 가. 아니오 \n- 이사회에서 기후변화나 탄소중립 외의...
2,3.0,이사회 내에서 정기적으로 환경경영 성과를 검토하고 있는가?,가. 아니오\n나. 비정기적\n다. 정기적,E,리더십과 거버넌스,거버넌스,-\n\n- (정의)\n1. 정기적: 아래 두 가지 요건 중 하나 이상 충족 필요 ...,다. 정기적,보고서 내에서 기후변화 위기 관리 프로세스를 비롯해 이사회 산하 ESG위원회 및 경...,"[36, 115, 110]",## 요약 \n- 등급: 다. 정기적\n\n## 상세 설명 \n\n1. 환경경영...
3,4.0,최고경영진의 환경경영 관련 책임 및 권한을 규정하고 명문화하였는가?,가. 아니오\n나. 경영진\n다. 최고경영진(C-Level),E,리더십과 거버넌스,거버넌스,-\n\n- (정의)\n1. 최고경영진(C-Level 임원): 직무 부문별 최고책임...,다. 최고경영진(C-Level),[풀이과정]\n1) 보고서 내 ‘지속가능경영 거버넌스’(9p) 및 ‘환경경영 체계’...,"[19, 110, 9]",```markdown\n## 요약 \n- 등급: 다. 최고경영진(C-Level) ...
4,5.0,환경경영 이행 전담조직을 구축하고 역할과 책임을 공개하고 있는가?,가. 아니오\n나. 예,E,리더십과 거버넌스,거버넌스,(확인사항)\n조직도 내 환경경영 전담 조직명 공개 혹은 환경경영 담당팀/담당자의 ...,가. 아니오,이미지 내 언급된 내용(예: 10페이지 등)을 보면 ‘ESG 전담 조직(ESG기획팀...,"[10, 100, 110]",## 요약 \n- 등급: 가. 아니오 \n- 환경경영 이행 전담조직의 구체적 역...
...,...,...,...,...,...,...,...,...,...,...,...
162,62.0,최근 정기주주총회의 의결권 행사 현황*을 홈페이지에 공개하고 있는가? * 참석 의결...,가. 아니오\n나. 예,G,이해관계자 소통,정보공개,(확인사항) \n- 최근 정기주주총회의 참석 의결권 수 홈페이지 공개 여부 \n\n...,가. 아니오,제공된 보고서 내용(특히 Governance 섹션)에서는 최근 정기주주총회의 참석 ...,"[86, 82, 13]",```markdown\n## 요약 \n- 등급: 가. 아니오\n\n## 상세 설명...
163,63.0,감사보고서 또는 사업보고서를 영문으로 공개하는가?,가. 아니오\n나. 예,G,이해관계자 소통,정보공개,(확인사항)\n- 최근 2개 사업연도까지의 영문 감사보고서 대외 공개 여부\n- 최...,가. 아니오,제공된 보고서 개요(124페이지) 및 ESG Factbook 등에서 영문 감사보고서...,"[32, 116, 124]",## 요약 \n - 등급: 가. 아니오\n - 영문 감사보고서 또는 사업보고서...
164,64.0,"최근 E,S,G 개별 등급 및 통합 ESG 평가등급에 관한 정보를 홈페이지에 공개하...",가. 아니오\n나. 예,G,이해관계자 소통,정보공개,"(확인사항) \n- 지배구조(G)평가등급 홈페이지 공개 여부 \n- 평가기관명, 평...",가. 아니오,이미지에 제시된 보고서 내용(ESG Factbook 및 인증/특허 현황 등)을 살펴...,"[110, 106, 111]","## 요약\n- 등급: 가. 아니오\n- 최근 E, S, G 개별 등급 및 통합 E..."
165,65.0,"최근 5개년의 배당정보*를 홈페이지에 공개하고 있는가? * 배당총액, 주당 배당금,...",가. 아니오\n나. 예,G,이해관계자 소통,정보공개,(정의) \n\n- 배당총액: 주식 총 발행 수량x주당배당금 \n\n\n\n(확인사...,가. 아니오,"(1) 제공된 이미지 내용을 보면, 배당 정보를 2021년, 2022년, 2023년...","[86, 98, 87]",## 요약 \n- 등급: 가. 아니오\n- 최근 5개년의 배당 정보를 홈페이지에 ...


In [94]:
g_df.loc[22, "question"]

'최근 3개년 온실가스(Scope1,2) 감축률'

In [91]:
md_content = g_df.loc[22, "rewrited_explanation_md"]

from IPython.display import display, Markdown

display(Markdown(md_content))


## 요약
- 최근 3개년(2021→2023) Scope 1, 2 온실가스 배출량은 2021년 2,384,071 tCO2eq, 2022년 2,404,069 tCO2eq, 2023년 2,275,751 tCO2eq로, 2021년 대비 약 4.5% 감축되었습니다.

## 상세 설명

1. **온실가스 배출량**
   - 2021년: 2,384,071 tCO2eq
   - 2022년: 2,404,069 tCO2eq
   - 2023년: 2,275,751 tCO2eq

2. **감축률**
   - 2021년 대비 2023년 온실가스 배출량이 약 4.5% 감소하였습니다.

3. **자료 출처**
   - 첨부된 ESG Factbook의 페이지 36과 38에서 온실가스 배출량(Scope 1+2)에 관한 데이터를 확인하였습니다.
   - 보고서는 각 연도별 구체적인 tCO2eq 수치를 제공하여 배출량 변화를 명확히 보여줍니다.

## 주요 근거 자료

- **36페이지**: Scope 1, 2 배출량에 대한 구체적인 수치와 연도별 비교를 제공합니다.
- **38페이지**: 온실가스 배출 목표와 실적을 비교하여 감축 노력을 설명합니다.

이 정보를 통해 현대자동차는 지속적인 온실가스 감축 노력을 하고 있음을 알 수 있으며, 이는 기업의 환경적 책임이 강화되고 있음을 나타냅니다.